### 1.	Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [152]:
import re
import random
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

random.seed(42)

In [153]:
PATH_WAR = "/kaggle/input/tolstoy-novels/WarAndPeace.txt"
PATH_ANNA = "/kaggle/input/tolstoy-novels/AnnaKarenina.txt"

UNKNOWN_SYMBOL = '&' # Символ для неизвестных соответствий

In [154]:
with open(PATH_WAR, 'r') as f_ru:
    war_and_peace = f_ru.read()

In [155]:
def preprocess_text(text):
    prep_text = re.sub("[^а-я ]", " ", text.lower())
    return re.sub(r'\s+', ' ',prep_text)

war_and_peace = preprocess_text(war_and_peace)

In [156]:
def train_test_split(text, n):
    train_text, test_text = text[:n], text[n:]
    return train_text, test_text

In [157]:
n = -10000

train_part_ru, test_part_ru = train_test_split(war_and_peace, n)
train_part_eng, test_part_eng = train_test_split(war_and_peace_eng, n)

In [158]:
char_cnt_ru = Counter(train_part_ru)
char_cnt_eng = Counter(train_part_eng)

In [159]:
chars_ru = list(char_cnt_ru.keys())
chars_ru_encoded = chars_ru.copy()
np.random.shuffle(chars_ru_encoded)
print(chars_ru)
print(chars_ru_encoded)
encode_mapping_ru = dict(zip(chars_ru, chars_ru_encoded))
decode_mapping_ru = {v: k for k, v in encode_mapping_ru.items()} # обратный маппинг для оценки

[' ', 'в', 'о', 'й', 'н', 'а', 'и', 'м', 'р', 'с', 'ы', 'з', 'е', 'т', 'л', 'ь', 'к', 'ч', 'г', 'д', 'у', 'п', 'я', 'ж', 'б', 'щ', 'ф', 'э', 'х', 'ю', 'ш', 'ц', 'ъ']
['ч', 'з', 'и', 'в', 'е', 'ъ', 'й', 'ь', 'а', 'ю', 'д', 'ш', 'т', 'э', 'ж', 'ы', 'р', 'н', 'к', 'щ', 'ф', 'м', ' ', 'б', 'г', 'я', 'п', 'ц', 'л', 'у', 'х', 'с', 'о']


In [160]:
def encode_text(text, mapping):
    text_encoded = ''
    for char in text:
        text_encoded += mapping.get(char, UNKNOWN_SYMBOL)
    return text_encoded

test_part_ru_encoded = encode_text(test_part_ru, encode_mapping_ru)

In [161]:
test_part_ru_encoded[:500]

'щзйефжъюычюбъжъюычэаиефжъюычетюрижыричхъкизчйчим эычиюэъеизйжъюычмаивэйчцэйчюэичхъкизчйчеъзтаеитчюмъютечмаиюэи эычтятчщзтчьйефэдчйчмикйгчеъзтаеитчщфьъжчръбщдвчщижилизчюэи зхйвчзчютатщйетчэижмдчазъефжю чрчраъучмжиэйедчюгйзчючеикчщзфлчюижщъэчйчюгтбъжчеъчюрижышрйвчжтщчмирадзхйвчмафщчюзиаънйзъвчшърайнъжчиечмищмадкйзъ чмичжыщфчриэиадвчэатяъжчмищчейьчюзиаънйзъвчрайнъжчиечеъчиафщйтчщтабйэчжтщчщтабъжчткичеичкефжю чйчэатяъжчйчинтзйщеичгджичнэичетчэижыричмищчиафщйтьчйжйчэижмивчеъаищъчеичмищчейьчищейьчиечю'

In [162]:
def decode(text, freq_mapping):
    char_cnt_encoded = Counter(text).most_common()
    freq_mapping = freq_mapping.most_common()
    mapping = {}
    for i, (char, _) in enumerate(char_cnt_encoded):
        mapping[char] = freq_mapping[i][0]
    text_decoded = ''
    for char in text:
        text_decoded += mapping.get(char, UNKNOWN_SYMBOL)
    return text_decoded, mapping

test_part_ru_decoded, freq_mapping_ru = decode(test_part_ru_encoded, char_cnt_ru)

In [163]:
test_part_ru_decoded[:500]

'двниптесз сшетесз лроиптесз иаскотзко жегов н оуялз ослеиовнтесз уройлн юлн сло жегов н иевариоа суесаи урослоялз аэа два мнипль н уогнб иевариоа дпмет кешдьй дотохов слоявжнй в сарадниа лотуь рвеиптся к крец утолниь сбнв с иог двпх сотдел н сбашет ие скотзыкнй тад уокрьвжнй урпд своречнвей ыекрнчет ои уодурьгнвея уо тздп колорьй лраэет уод инм своречнвей крнчет ои ие орпдна даршнл тад даршет аго ио гиптся н лраэет н очавндио бьто чло иа лотзко уод орпднам нтн лотуой иероде ио уод инм одинм ои с'

In [164]:
def get_accuracy(freq_mapping, decode_mapping):
    correctly_mapped_chars = sum(1 for char in freq_mapping.keys() if freq_mapping[char] == decode_mapping[char])
    total_chars = len(freq_mapping)
    return correctly_mapped_chars / total_chars

In [168]:
print(f"Доля верно расшифрованных символов: {get_accuracy(freq_mapping_ru, decode_mapping_ru)}")

Доля верно расшифрованных символов: 0.42424242424242425


**2.	Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:**
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм (В качестве естественной метрики качества можно взять долю правильно расшифрованных букв или, если хочется совсем математически изощриться, расстояние между двумя перестановками, правильной и полученной из модели; но, честно говоря, в этом задании следить за численными метриками не так уж обязательно, будет и глазами всё видно).

In [169]:
def count_bigrams(text):
    bigrams = [text[i:i + 2] for i in range(0, len(text) - 1, 2)]
    bigram_counts = {}
    for bigram in bigrams:
        bigram_counts[bigram] = bigram_counts.get(bigram, 0) + 1
    return bigram_counts

bigram_cnt_ru = count_bigrams(train_part_ru)

In [170]:
test_part_ru_encoded[:500]

'щзйефжъюычюбъжъюычэаиефжъюычетюрижыричхъкизчйчим эычиюэъеизйжъюычмаивэйчцэйчюэичхъкизчйчеъзтаеитчюмъютечмаиюэи эычтятчщзтчьйефэдчйчмикйгчеъзтаеитчщфьъжчръбщдвчщижилизчюэи зхйвчзчютатщйетчэижмдчазъефжю чрчраъучмжиэйедчюгйзчючеикчщзфлчюижщъэчйчюгтбъжчеъчюрижышрйвчжтщчмирадзхйвчмафщчюзиаънйзъвчшърайнъжчиечмищмадкйзъ чмичжыщфчриэиадвчэатяъжчмищчейьчюзиаънйзъвчрайнъжчиечеъчиафщйтчщтабйэчжтщчщтабъжчткичеичкефжю чйчэатяъжчйчинтзйщеичгджичнэичетчэижыричмищчиафщйтьчйжйчэижмивчеъаищъчеичмищчейьчищейьчиечю'

In [171]:
def decode_text_by_bigrams(text, bigram_freq_mapping):
    bigrams = [text[i:i + 2] for i in range(0, len(text), 2)]
    
    mapping = {}
    for bigram, (symbol, _) in zip(bigrams, bigram_freq_mapping.items()):
        mapping[bigram] = symbol
    
    text_decoded = ''.join(mapping.get(bigram, '') for bigram in bigrams)
    
    return text_decoded

test_part_ru_decoded = decode_text_by_bigrams(test_part_ru_encoded, bigram_cnt_ru)

In [172]:
test_part_ru_decoded[:500]

'ждюжьи рьгирйе рьгжеф ьи рьгюсычсфа эпууъедблзыдшоьг ытябсюшбнвгкгптпрлзедлзштэпууъедблзуцшкжаьбшспямедшса ынпшоьгывйрждйркнздыжсгкгдаечлд ьддбсйрззяетгяюо пэьшсфер йшснпзамюжбдбмехмжююсбцсфюм эсъздбчнзтжнфбщкгвщкажсшсвздбьобсшежду шссфцлсблзвхстйелдтзычсфонуфжблбгмншнф бмюжбсакшшсыуьщмксъжбжмнфкдйелмдшншйчсытьсънзншгшлезсвлнпсыжбжеывйекгмглдфашсыуьщмксъжбнфкдйелмдшуцлмнвжюйршахр ягшдзьшддлвтгхнэпбсбкздбчнзлзжеывйесглмрпюшщ эппфвщккнплдйрнпумвлкгмглмнвжюднсгжжбцсфншжбуцптцллдэпншгмпшюнмгпшюнф шс'

In [173]:
def get_accuracy_(encoded_text, decoded_text, decode_mapping):
    mapping = {}
    for i in range(len(decoded_text)):
        if encoded_text[i] not in mapping:
            mapping[encoded_text[i]] = decoded_text[i]
    total_chars = len(mapping)
    correctly_mapped_chars = sum(1 for k in mapping.keys() if mapping[k] == decode_mapping[k])
    return correctly_mapped_chars / total_chars

In [174]:
print(f"Доля верно расшифрованных символов: {get_accuracy_(test_part_ru_encoded, test_part_ru_decoded, decode_mapping_ru)}%")

Доля верно расшифрованных символов: 0.030303030303030304%


Декодировка текста с использованием биграмм показывает результаты похуже по сравнению с декодированием на отдельных символах. Возможная причина - в тестовом корпусе присутствует больше различных комбинаций биграмм, чем отдельных символов

### 3.	Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

### Метод обучения перестановки символов на основе MCMC-сэмплирования с использованием статистики биграмм

Общая идея: изначально случайным образом выбирается перестановка символов. На каждой итерации алгоритма генерируется кандидат в новое состояние путем обмена местами двух случайно выбранных символов в текущей перестановке. Затем вычисляются вероятности встретить биграмму в корпусе для текста, расшифрованного с использованием новой перестановки и текущей перестановки.

Вероятность принятия новой перестановки определяется сравнением произведений вероятностей биграмм для нового и текущего состояний. Если произведение вероятностей для нового состояния больше или равно произведения вероятностей для текущего состояния, новая перестановка принимается. В противном случае, новая перестановка принимается с вероятностью, пропорциональной отношению вероятностей.

Процесс запускается несколько раз. После завершения MCMC-сэмплирования выбирается перестановка символов с наибольшим логарифмом правдоподобия, представляющая оптимальноую декодировку

In [131]:
def get_bigrams(text):
    return [text[i:i + 2] for i in range(0, len(text) - 1)]

def calc_log_likelihood(decoded_text, train_freqs):
    total_bigrams = sum(train_freqs.values())
    log_likelihood = 0
    for i in range(len(decoded_text) - 1):
        bigram = decoded_text[i:i+2]
        log_likelihood += np.log((train_freqs[bigram] + 1) / (total_bigrams + 1))
    return log_likelihood

def random_swap_chars(chars_list):
    chars_list_local = chars_list.copy()
    ids_to_swap = np.random.choice(len(chars_list_local), size=2, replace=False)
    chars_list_local[ids_to_swap[0]], chars_list_local[ids_to_swap[1]] = chars_list_local[ids_to_swap[1]], chars_list_local[ids_to_swap[0]]
    return chars_list_local

def decode_text_by_mapping(text, mapping):
    text_decoded = ''.join(mapping.get(char, UNKNOWN_SYMBOL) for char in text)
    return text_decoded

def get_ordered_chars(text):
    return [char for char, _ in Counter(text).most_common()]

def mcmc(train_part, test_encoded, iters, print_n):
    best_score = -np.inf
    best_mapping = None
    train_chars_list = get_ordered_chars(train_part)
    test_chars_list = get_ordered_chars(test_encoded)

    train_bigrams = get_bigrams(train_part)
    train_bigrams_cnt = Counter(train_bigrams)

    mapping = dict(zip(test_chars_list, train_chars_list))
    text_decoded = decode_text_by_mapping(test_encoded, mapping)
    score = calc_log_likelihood(text_decoded, train_bigrams_cnt)
    trained_chars = train_chars_list.copy()

    for i in tqdm(range(iters)):
        trained_chars_new = random_swap_chars(trained_chars)
        mapping_new = dict(zip(test_chars_list, trained_chars_new))
        text_decoded_new = decode_text_by_mapping(test_encoded, mapping_new)
        score_new = calc_log_likelihood(text_decoded_new, train_bigrams_cnt)

        score_diff = np.exp(score_new - score)
        if score_diff > np.random.uniform(0, 1):
            score = score_new
            mapping = mapping_new
            trained_chars = trained_chars_new.copy()

        if score > best_score:
            best_score = score
            best_mapping = mapping

        if i % print_n == 0:
            print(f'Iter: {i}, best_score: {best_score}')
            res = decode_text_by_mapping(test_encoded, best_mapping)
            print(res[:300])

    return best_mapping


In [132]:
%%time
mcmc_mapping_ru = mcmc(train_part_ru, test_part_ru_encoded, 10000, 1000)

  0%|          | 12/10000 [00:00<02:46, 59.86it/s]

Iter: 0, best_score: -66850.31549431116
двниптесз сшетесз лроиптесз иаскотзко жегов н оуялз ослеиовнтесз уройлн юлн сло жегов н иевариоа суесаи урослоялз аэа два мнипль н уогнб иевариоа дпмет кешдьй дотохов слоявжнй в сарадниа лотуь рвеиптся к крец утолниь сбнв с иог двпх сотдел н сбашет ие скотзыкнй тад уокрьвжнй урпд своречнвей ыекрнчет


 10%|█         | 1008/10000 [00:16<02:37, 57.22it/s]

Iter: 1000, best_score: -56687.30003514354
дтинулась сжалась вронулась несколько шагот и опявь осванотилась пройви цви сво шагот и натерное спасен просвоявь еще дте минувы и погиб натерное думал каждый долохот своятший т середине волпы ртанулся к краю пловины сбит с ног дтух солдав и сбежал на скольчкий лед покрытший пруд сторазитай чакризал


 20%|██        | 2008/10000 [00:33<02:12, 60.53it/s]

...

Iter: 8000, best_score: -55004.2988246911
двинулась сжалась тронулась несколько шагов и опять остановилась пройти эти сто шагов и наверное спасен простоять еще две минуты и погиб наверное думал каждый долохов стоявший в середине толпы рванулся к краю плотины сбив с ног двух солдат и сбежал на скользкий лед покрывший пруд сворачивай закричал


 90%|█████████ | 9012/10000 [02:29<00:16, 60.94it/s]

Iter: 9000, best_score: -55004.2988246911
двинулась сжалась тронулась несколько шагов и опять остановилась пройти эти сто шагов и наверное спасен простоять еще две минуты и погиб наверное думал каждый долохов стоявший в середине толпы рванулся к краю плотины сбив с ног двух солдат и сбежал на скользкий лед покрывший пруд сворачивай закричал


100%|██████████| 10000/10000 [02:46<00:00, 60.17it/s]

CPU times: user 2min 46s, sys: 760 ms, total: 2min 46s
Wall time: 2min 46s


In [175]:
test_part_ru_decoded_mcmc = decode_text_by_mapping(test_part_ru_encoded, mcmc_mapping_ru)
print(f"Доля верно расшифрованных символов: {get_accuracy_(test_part_ru_encoded, test_part_ru_decoded_mcmc, decode_mapping_ru)}")

Доля верно расшифрованных символов: 0.06060606060606061


In [138]:
test_part_ru_decoded_mcmc[:500]

'двинулась сжалась тронулась несколько шагов и опять остановилась пройти эти сто шагов и наверное спасен простоять еще две минуты и погиб наверное думал каждый долохов стоявший в середине толпы рванулся к краю плотины сбив с ног двух солдат и сбежал на скользкий лед покрывший пруд сворачивай закричал он подпрыгивая по льду который трещал под ним сворачивай кричал он на орудие держит лед держал его но гнулся и трещал и очевидно было что не только под орудием или толпой народа но под ним одним он с'

### 4.	Расшифруйте сообщение:<br>
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
<br><br>
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):<br>
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ
<br><br>

In [143]:
text_encoded = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [148]:
with open(PATH_WAR, 'r') as f_ru_1, open(PATH_ANNA, 'r') as f_ru_2:
    war_and_peace = f_ru_1.read()
    anna_karenina = f_ru_2.read()
    corpus_rus = war_and_peace + ' ' + anna_karenina

corpus_rus = preprocess_text(corpus_rus)

In [149]:
%%time
mcmc_mapping = mcmc(corpus_rus, text_encoded, 500000, 100000)

  0%|          | 241/500000 [00:00<03:27, 2406.79it/s]

Iter: 0, best_score: -1616.5060196213333
олна рд раяасо иевьтнуидг ана кемса иевьтнуидг соплс б шсеые леезжоиач песевдг ноыпе квемастсу лпевоо рлоые рд рло ляонтна квтрануие а кенбмасо ьтплаьтнуидг зтнн йт келнояиоо мосровсео йтятиао пбвлт хесч пеиомие ч иамоые ио езожтю


 20%|██        | 100483/500000 [00:43<02:50, 2337.93it/s]

Iter: 100000, best_score: -1241.5561722197178
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


 40%|████      | 200700/500000 [01:26<02:06, 2375.10it/s]

...

Iter: 300000, best_score: -1241.5561722197178
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


 80%|████████  | 400528/500000 [02:52<00:42, 2368.28it/s]

Iter: 400000, best_score: -1241.5561722197178
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


100%|██████████| 500000/500000 [03:35<00:00, 2317.65it/s]

CPU times: user 3min 36s, sys: 5.04 s, total: 3min 41s
Wall time: 3min 36s


In [150]:
text_encoded

'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [151]:
text_decoded_mcmc = decode_text_by_mapping(text_encoded, mcmc_mapping)
text_decoded_mcmc

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

### 6. Бонус

- **Криптография:** Эти модели можно использовать для расшифровки закодированных сообщений

- **NLP:** Исправление орфографических ошибок; определние языка (русский, английский или другой) - для разных языков будет разная частотность, анализ больших текстов: выявление закономерностей и тенденций

- **Сжатие данных:** например, алгоритм BPE

- **Машинное обучение:** Т к у разных текстов разная частотность n-грамм, то можно сопоставлять и находить похожие тексты по этим частотностям (если counter-ы похожи, то тексты одинаковые) - тематическое моделирование